In [1]:
import os
os.environ['TP_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1,28*28).astype('float32')/255.0
x_test = x_test.reshape(-1,28*28).astype('float32')/255.0

In [3]:
class Dense(layers.Layer):
    def __init__(self,units):
        super(Dense,self).__init__()
        self.units = units 

    def build(self,input_shape):
        self.w = self.add_weight(
            name= 'w', shape = (input_shape[-1],self.units),
            initializer = 'random_normal',
            trainable = True,
        )
        self.b = self.add_weight(
            name= 'b', shape = (self.units,),
            initializer = 'zeros',
            trainable = True,
        )

    def call(self,inputs):
        return tf.matmul(inputs, self.w)+self.b

In [4]:
class MyRelu(layers.Layer):
    def __init__(self):
        super(MyRelu,self).__init__()

    def call(self,x):
        return tf.math.maximum(x,0)

In [5]:
class MyModel(keras.Model):
    def __init__ (self,num_classes = 10):
        super(MyModel,self).__init__()
        self.dense1 = Dense(64)
        self.dense2 = Dense(num_classes)
        self.relu = MyRelu()
        # self.dense1 = layers.Dense(64)
        # self.dense2 = layers.Dense(num_classes)

    def call(self,input_tensor):
        # x= tf.nn.relu(self.dense1(input_tensor))
        x= self.relu(self.dense1(input_tensor))
        return self.dense2(x)
    
model = MyModel()

In [6]:
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy'],
    run_eagerly = True,
    )

In [7]:
model.fit(x_train, y_train, batch_size = 32 , epochs = 10, verbose = 2)
print(model.summary())
model.evaluate(x_test, y_test, batch_size = 32, verbose = 2)

Epoch 1/10
1875/1875 - 19s - loss: 0.3474 - accuracy: 0.9038 - 19s/epoch - 10ms/step
Epoch 2/10
1875/1875 - 18s - loss: 0.1655 - accuracy: 0.9518 - 18s/epoch - 10ms/step
Epoch 3/10
1875/1875 - 19s - loss: 0.1177 - accuracy: 0.9657 - 19s/epoch - 10ms/step
Epoch 4/10
1875/1875 - 18s - loss: 0.0908 - accuracy: 0.9727 - 18s/epoch - 9ms/step
Epoch 5/10
1875/1875 - 17s - loss: 0.0746 - accuracy: 0.9770 - 17s/epoch - 9ms/step
Epoch 6/10
1875/1875 - 17s - loss: 0.0621 - accuracy: 0.9811 - 17s/epoch - 9ms/step
Epoch 7/10
1875/1875 - 17s - loss: 0.0529 - accuracy: 0.9839 - 17s/epoch - 9ms/step
Epoch 8/10
1875/1875 - 17s - loss: 0.0452 - accuracy: 0.9862 - 17s/epoch - 9ms/step
Epoch 9/10
1875/1875 - 18s - loss: 0.0391 - accuracy: 0.9882 - 18s/epoch - 9ms/step
Epoch 10/10
1875/1875 - 18s - loss: 0.0334 - accuracy: 0.9899 - 18s/epoch - 9ms/step
Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense

[0.08791892975568771, 0.9750000238418579]